In [66]:
test_input = [int(e) for e in list("389125467")]

test_result = 67384529

In [67]:
import itertools


def flatten(x):
    
    res = []
    
    for e in x:
        
        if isinstance(e, int):
            res.append(e)
        elif isinstance(e, list):
            for i in flatten(e):
                res.append(i)
        else:
            print("shouldn't happen")
            
    return res

In [68]:
flatten([1, 2, [4, 5], 3])

[1, 2, 4, 5, 3]

In [69]:
cups = [int(e) for e in list("963275481")]

In [70]:
cups

[9, 6, 3, 2, 7, 5, 4, 8, 1]

Game = 100 turns

In [71]:
n_turns = 100

The crab picks up the three cups that are immediately clockwise of the current cup.

They are removed from the circle; cup spacing is adjusted as necessary to maintain the circle.

In [65]:
cups[cups.index(6)+1:]

[3, 2, 7, 5, 4, 8, 1]

In [94]:
def pick_up_cups(idx, cups):

    cups = cups.copy()

    current = cups[idx]

    picked_up = []

    to_pick = idx + 1

    for i in range(3):

        if (to_pick >= len(cups)) or (cups[to_pick] == current):
            to_pick = 0

        # which_to_pick = cups[to_pick]

        #         if which_to_pick == current:
        #             # wrap
        #             to_pick = 0

        picked_up.append(cups.pop(to_pick))

    #     adj = cups[1:4]

    return current, picked_up, cups

In [95]:
cups

[9, 6, 3, 2, 7, 5, 4, 8, 1]

In [100]:
pick_up_cups(6, cups)

(4, [8, 1, 9], [6, 3, 2, 7, 5, 4])

The crab selects a destination cup: the cup with a label equal to the current cup's label minus one. If this would select one of the cups that was just picked up, the crab will keep subtracting one until it finds a cup that wasn't just picked up. If at any point in this process the value goes below the lowest value on any cup's label, it wraps around to the highest value on any cup's label instead.

In [128]:
def choose_destination(chosen, picked_cups, cups):

    destination_cup_label = chosen - 1

    if destination_cup_label < min(cups):
        destination_cup_label = max(cups)

    while destination_cup_label in picked_cups:

        new_val = destination_cup_label - 1

        if new_val < min(cups):
            new_val = max(cups)

        destination_cup_label = new_val

    return destination_cup_label

The crab places the cups it just picked up so that they are immediately clockwise of the destination cup. They keep the same order as when they were picked up.

In [129]:
def insert_in_pos(l: list, pos: int, elems: list) -> list:
    
    l = l.copy()
    l.insert(pos, elems)
    

    return list(flatten(l))

In [130]:
insert_in_pos([1,2,3], 2, [4,5])

[1, 2, 4, 5, 3]

The crab selects a new current cup: the cup which is immediately clockwise of the current cup.

In [568]:
import itertools

def getn(cups):

    res = []
    
    seen = False

    for x in itertools.cycle(cups):

        if seen:
            res.append(str(x))

        if x == 1:
            seen = True
            
        if len(res) == 8:
            break
            
    return "".join(res)

In [569]:
test_input

[3, 8, 9, 1, 2, 5, 4, 6, 7]

In [584]:
%%time
game_cups = cups.copy()

idx_start = 0

verbose = False

for i in range(n_turns):

    chosen, picked, left = pick_up_cups(idx_start, game_cups)

    dest = choose_destination(chosen, picked, game_cups)

    dest_idx = left.index(dest) + 1

    if verbose:
        print(
            f"""
    -- Move {i} --
        cups: {game_cups}
        chosen: {chosen}
        pick up: {picked}
        destination: {dest}
        dest_idx: {dest_idx}
        """
        )

    game_cups = insert_in_pos(left, dest_idx, picked)

    prev_cup_idx = game_cups.index(chosen)

    idx_start = prev_cup_idx + 1

    if idx_start >= len(game_cups):
        idx_start = 0

CPU times: user 3.57 ms, sys: 36 µs, total: 3.61 ms
Wall time: 3.79 ms


In [585]:
getn(game_cups) # 97632548

'95487632'

## Part 2

In [1]:
cups = [int(e) for e in list("963275481")]

extra = list(range(max(cups) + 1, 1_000_000 + 1))

cups += extra

Double linked list

dict {num: next_num}

In [2]:
class RotationList:
    def __init__(self, values: list):

        # self.d = dict(enumerate(values))

        self.d = {}

        for idx, val in enumerate(values):

            n = idx + 1

            if n >= len(values):
                n = 0

            self.d[val] = {"prev": values[idx - 1], "next": values[n]}

        self.total_len = len(values)

        self.max_val = max(values)
        self.min_val = min(values)

    def pick_up_cups(self, value):

        n1 = self.d[value]["next"]
        n2 = self.d[n1]["next"]
        n3 = self.d[n2]["next"]

        nn = self.d[n3]["next"]

        self.d[value]["next"] = nn

        return value, (n1, n2, n3)

    def choose_destination(self, chosen, picked_cups):

        destination_cup = chosen - 1

        if destination_cup < self.min_val:
            destination_cup = self.max_val

        while destination_cup in picked_cups:

            new_val = destination_cup - 1

            if new_val < self.min_val:
                new_val = self.max_val

            destination_cup = new_val

        return destination_cup

    def insert_in_pos(self, pos: int, elems: list):

        assert len(elems) == 3

        old_nex = self.d[pos]["next"]
        self.d[pos]["next"] = elems[0]
        self.d[elems[-1]]["next"] = old_nex

    def next_idx(self, prev_chosen):

        return self.d[prev_chosen]["next"]

    def score_part_2(self):

        n1 = self.d[1]["next"]
        n2 = self.d[n1]["next"]

        return n1 * n2

OO double linked list approach (lots of self mutations)

In [3]:
game_cups = RotationList(cups.copy() + list(range(max(cups) + 1, 1000000 + 1)))

In [4]:
n_turns = 10_000_000

[Review](https://topaz.github.io/paste/#XQAAAQDGAgAAAAAAAAAyGUj/T32X5leZ5SDz2qdpTIjS9oXYfUdcfc/VCSARNAsWyShkROB2xn+j7UcC2v0l879pAkcPfa+DiiCE8r019XB0dWpdi6WkXAQXUljZ5IycPeN+r1XvKj5o20hnd1x1S7/vzVUd3buLgY2gljaXs1dqGM2DvjiN8YE1HVGinjUQQD5oYiGgImU3ZzFPBNHrb7SSJhmH0qm48zpNQ/KHCmll+KD8dYQhLWQyQTmCtSkY2y8lxTIsfP21IqhQgVQRK97ojghdiK7d6FWH/w3bChWIb0IKhzxTn5K2spuSwbqsnp/m5OR5Y8Js8oaAIGrmdGGqEq6FkdyggUncrLbb1q8BtzvjnVwG+SeJGPodczKzObJhqCsEsRfLa2djXbotjlCJOegrqXNNoJtv/qkxlw==)

In [5]:
val = 9

verbose = False

for i in range(n_turns):

    if i % 1_000_000 == 0:
        print(i)

    current, picked_up = game_cups.pick_up_cups(val)

    dest = game_cups.choose_destination(current, picked_up)

    game_cups.insert_in_pos(dest, picked_up)

    val = game_cups.next_idx(current)


print("Result", game_cups.score_part_2())  # 412990492266

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
412990492266
